## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
import ipywidgets as widgets
widgets.IntSlider()

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
file_path = 'Resources/WeatherPy_Database.csv'
#path = os.path.abspath(csv)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
#file_to_load = os.path.join('...Weather_Database','WeatherPy_database.csv')
city_data_df = pd.read_csv(file_path)
#with open(file_to_load) as city_data_df:
 #   file_reader = csv.reader(city_data_df)
  #  headers = next(file_reader)
    
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Alice springs,AU,-23.70,133.88,87.80,12,0,10.29,clear sky
1,Parksville,CA,49.32,-124.32,48.20,87,21,9.17,few clouds
2,Marbella,ES,36.52,-4.89,66.99,68,0,2.24,clear sky
3,Sur,OM,22.57,59.53,83.64,22,0,7.81,clear sky
4,Bedi,IN,22.50,70.05,89.40,26,0,11.83,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input('What is the maximum temperature you would like for your trip? '))
min_temp = float(input('What is the minimum temperature you would like for your trip? '))

What is the maximum temperature you would like for your trip? 95
What is the minimum temperature you would like for your trip? 64


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Alice springs,AU,-23.70,133.88,87.80,12,0,10.29,clear sky
2,Marbella,ES,36.52,-4.89,66.99,68,0,2.24,clear sky
3,Sur,OM,22.57,59.53,83.64,22,0,7.81,clear sky
4,Bedi,IN,22.50,70.05,89.40,26,0,11.83,clear sky
5,Padang,ID,-0.95,100.35,80.82,73,100,4.03,light rain
6,Mogadishu,SO,2.04,45.34,86.00,62,40,13.87,scattered clouds
9,Port alfred,ZA,-33.59,26.89,69.84,69,32,23.82,light rain
10,Ponta do sol,PT,32.67,-17.10,77.00,36,20,4.70,few clouds
14,Busselton,AU,-33.65,115.33,64.00,63,100,1.16,light rain
15,Puerto del rosario,ES,28.50,-13.86,75.20,78,0,16.11,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Alice springs,AU,87.80,clear sky,-23.70,133.88,
2,Marbella,ES,66.99,clear sky,36.52,-4.89,
3,Sur,OM,83.64,clear sky,22.57,59.53,
4,Bedi,IN,89.40,clear sky,22.50,70.05,
5,Padang,ID,80.82,light rain,-0.95,100.35,
6,Mogadishu,SO,86.00,scattered clouds,2.04,45.34,
9,Port alfred,ZA,69.84,light rain,-33.59,26.89,
10,Ponta do sol,PT,77.00,few clouds,32.67,-17.10,
14,Busselton,AU,64.00,light rain,-33.65,115.33,
15,Puerto del rosario,ES,75.20,clear sky,28.50,-13.86,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotel_df.loc[index,'Hotel name']= hotels['results'][0]['name']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

IndexError: list index out of range

In [10]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df
#hotel_df.dropna()
#hotel_df.rest_index(drop = True)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name,Hotel name
0,Alice springs,AU,87.80,clear sky,-23.70,133.88,Desert Palms Alice Springs,Desert Palms Alice Springs
2,Marbella,ES,66.99,clear sky,36.52,-4.89,Hotel Fuerte Marbella,Hotel Fuerte Marbella
3,Sur,OM,83.64,clear sky,22.57,59.53,Sur Plaza Hotel,Sur Plaza Hotel
4,Bedi,IN,89.40,clear sky,22.50,70.05,Hotel Aram,Hotel Aram
5,Padang,ID,80.82,light rain,-0.95,100.35,Mercure Padang,Mercure Padang
6,Mogadishu,SO,86.00,scattered clouds,2.04,45.34,Hotel Juba,Hotel Juba
9,Port alfred,ZA,69.84,light rain,-33.59,26.89,The Halyards Hotel,The Halyards Hotel
10,Ponta do sol,PT,77.00,few clouds,32.67,-17.10,Hotel do Campo,Hotel do Campo
14,Busselton,AU,64.00,light rain,-33.65,115.33,Observatory Guest House,Observatory Guest House
15,Puerto del rosario,ES,75.20,clear sky,28.50,-13.86,Hotel JM Puerto del Rosario,Hotel JM Puerto del Rosario


In [13]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
                                         
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))